In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
!conda install -c conda-forge geocoder --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder# to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India').text

In [3]:
soup = BeautifulSoup(data,'html.parser')
neighborslist=[]
for row in soup.find_all("div",class_="mw-category")[0].findAll('li'):
    neighborslist.append(row.text)
    
hyd_df= pd.DataFrame({'neighborslist':neighborslist})
hyd_df.head()

,neighborslist
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet


In [4]:
hyd_df.shape

(200, 1)

In [5]:
def get_lonlat(neighborslist):
    lat_lon =None
    while (lat_lon is None):
        g= geocoder.arcgis('{}, Hyderabad ,India'.format(neighborslist))
        lat_lon=g.latlng
    return lat_lon

In [10]:
coords = [get_lonlat(neighborslist) for neighborslist in hyd_df['neighborslist']]

In [7]:
coords

[[17.411200000000065, 78.50824000000006],
 [17.392977027745946, 78.45686724051741],
 [17.337650000000053, 78.56414000000007],
 [17.389800000000037, 78.47658000000007],
 [17.410610000000077, 78.51513000000006],
 [17.37751000000003, 78.48005000000006],
 [17.387364823969637, 78.4669870622138],
 [17.34259000000003, 78.47626000000008],
 [17.36068000000006, 78.47998000000007],
 [17.503370000000075, 78.41602000000006],
 [17.535430000000076, 78.54427000000004],
 [17.385820000000024, 78.51836000000003],
 [17.435350000000028, 78.44861000000003],
 [17.40784000000002, 78.49150000000003],
 [17.385140000000035, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.40710000000007, 78.50233000000003],
 [17.372720000000072, 78.49047000000007],
 [17.38897000000003, 78.48681000000005],
 [17.39931000000007, 78.49964000000006],
 [17.339920000000063, 78.54553000000004],
 [17.448510000000056, 78.44924000000003],
 [17.415350000000046, 78.43435000000005],
 [17.38859199570786, 78.47665099785392],
 

In [8]:
df_coords =pd.DataFrame(data=coords,columns=['Latitude','Longitude'])

In [9]:
df_coords.head()

,Latitude,Longitude
0,17.411200,78.508240
1,17.392977,78.456867
2,17.337650,78.564140
3,17.389800,78.476580
4,17.410610,78.515130


In [11]:
hyd_df['Latitude']=df_coords['Latitude']
hyd_df['Longitude']=df_coords['Longitude']
hyd_df.head()

,neighborslist,Latitude,Longitude
0,A. S. Rao Nagar,17.411200,78.508240
1,A.C. Guards,17.392977,78.456867
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adikmet,17.410610,78.515130


In [12]:
hyd_df.to_csv('hyd_data.csv',index=False)

In [13]:
address='Hyderabad,India'

geolocator=Nominatim(user_agent="my application")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('latitude and longitude of hyderabad are {} , {}.'.format(latitude , longitude))

latitude and longitude of hyderabad are 17.38878595 , 78.46106473453146.


In [14]:
map_hyd =folium.Map(location=[latitude,longitude],zoom_start=10)
map_hyd

In [15]:

map_hyd=folium.Map(location=[latitude,longitude],zoom_start=11)

for lat,lng,label in zip(hyd_df["Latitude"],hyd_df["Longitude"],hyd_df["neighborslist"]):
    label='{}'.format("neighborslist")
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],popup=label,radius=5,color='blue',fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_hyd)
    
map_hyd

In [16]:
map_hyd.save('map_hyd.html')

In [33]:
# define Foursquare Credentials and Version
CLIENT_ID = 'SNJEDLSCSCISUYSLX315BOQZDCQ1R0BYDSBVY40TQHMXKCSU' # your Foursquare ID
CLIENT_SECRET = 'DL233VQ0JBF3W3Q03SA005SB0FD2TIXLLYMG5SJOZU1B2H4H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: SNJEDLSCSCISUYSLX315BOQZDCQ1R0BYDSBVY40TQHMXKCSU
CLIENT_SECRET:DL233VQ0JBF3W3Q03SA005SB0FD2TIXLLYMG5SJOZU1B2H4H


In [35]:
LIMIT = 100
radius = 2000

venues = []

for lat, long, neighborhood in zip(hyd_df['Latitude'], hyd_df['Longitude'], hyd_df['neighborslist']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [36]:
venues_df = pd.DataFrame(venues,columns=['Neighborhood','Latitude','Longitude','venuename','venueLatitude','venueLongitude','venuecategory'])
venues_df.head()

,Neighborhood,Latitude,Longitude,venuename,venueLatitude,venueLongitude,venuecategory
0,A. S. Rao Nagar,17.4112,78.50824,Bawarchi,17.406369,78.497662,Indian Restaurant
1,A. S. Rao Nagar,17.4112,78.50824,Subway,17.404173,78.514950,Sandwich Place
2,A. S. Rao Nagar,17.4112,78.50824,Sudharshan Theatre 35mm,17.406530,78.495150,Movie Theater
3,A. S. Rao Nagar,17.4112,78.50824,Devi 70 MM,17.406329,78.495409,Movie Theater
4,A. S. Rao Nagar,17.4112,78.50824,Baskin-Robbins,17.404311,78.510034,Ice Cream Shop


In [37]:
venues_df['venuecategory'].unique()

array(['Indian Restaurant', 'Sandwich Place', 'Movie Theater',
       'Ice Cream Shop', 'Coffee Shop', 'Convenience Store', 'Café',
       'Asian Restaurant', 'Pharmacy', 'Flea Market', 'Bakery',
       'Grocery Store', 'Park', 'Hyderabadi Restaurant', 'Juice Bar',
       'Lounge', 'South Indian Restaurant', 'Bistro', 'Science Museum',
       'Middle Eastern Restaurant', 'Snack Place', 'Hotel Bar',
       'Performing Arts Venue', 'Vegetarian / Vegan Restaurant',
       'Stadium', 'Hotel', 'Pizza Place', 'Fast Food Restaurant',
       'Mobile Phone Shop', 'Department Store', 'Fried Chicken Joint',
       'Hookah Bar', 'Electronics Store', 'Clothing Store',
       'Chinese Restaurant', 'Pool', 'Restaurant',
       'Fruit & Vegetable Store', 'Shoe Store', 'Dessert Shop', 'Diner',
       'Food Truck', 'Chaat Place', 'Neighborhood', 'Burger Joint',
       'Multiplex', 'Smoke Shop', 'Breakfast Spot', 'Bar',
       'Shopping Mall', 'Food', 'Indie Movie Theater', 'Gaming Cafe',
       'Farmers

In [38]:
venues_df.groupby('Neighborhood').count()

,Latitude,Longitude,venuename,venueLatitude,venueLongitude,venuecategory
Neighborhood,,,,,,
A. S. Rao Nagar,23,23,23,23,23,23
A.C. Guards,55,55,55,55,55,55
Abhyudaya Nagar,11,11,11,11,11,11
Abids,80,80,80,80,80,80
Adikmet,22,22,22,22,22,22
Afzal Gunj,47,47,47,47,47,47
Aghapura,55,55,55,55,55,55
"Aliabad, Hyderabad",10,10,10,10,10,10
Alijah Kotla,15,15,15,15,15,15


In [41]:
'Restaurant' in venues_df['venuecategory'].unique()

True

In [42]:
hyd_shop = pd.get_dummies(venues_df['venuecategory'],prefix="",prefix_sep='')
hyd_shop['Neighborhoods']=venues_df['Neighborhood']
fixed_columns = [hyd_shop.columns[-1]] + list(hyd_shop.columns[:-1])
hyd_shop = hyd_shop[fixed_columns]

print(hyd_shop.shape)
hyd_shop.head()

(6717, 169)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Campground,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [43]:
len(hyd_grouped[hyd_grouped['Restaurant']>0])

18

In [48]:
hyd =hyd_grouped[['Neighborhoods','Restaurant']]
hyd.head()

,Neighborhoods,Restaurant
0,A. S. Rao Nagar,0.0
1,A.C. Guards,0.0
2,Abhyudaya Nagar,0.0
3,Abids,0.0
4,Adikmet,0.0


In [50]:
k =3
hyd_clust=hyd_grouped.drop(["Neighborhoods"],axis=1)
kmeans =KMeans(n_clusters=k,init='k-means++',random_state=0).fit(hyd_clust)

In [51]:
kmeans.labels_

array([2, 2, 2, 2, 1, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2])

In [52]:
hyd_copy =hyd.copy()
hyd_copy['cluster labels']= kmeans.labels_
hyd_copy.rename(columns={"Neighborhoods":"Neighborhood"},inplace=True)
hyd_copy.head()

,Neighborhood,Restaurant,cluster labels
0,A. S. Rao Nagar,0.0,2
1,A.C. Guards,0.0,2
2,Abhyudaya Nagar,0.0,2
3,Abids,0.0,2
4,Adikmet,0.0,1


In [53]:
hyd_copy['Latitude']=hyd_df['Latitude']
hyd_copy['Longitude']= hyd_df['Longitude']
hyd_copy.shape

(181, 5)

In [54]:
hyd_copy.sort_values(["cluster labels"], inplace=True)
hyd_copy.head()

,Neighborhood,Restaurant,cluster labels,Latitude,Longitude
134,Nawab Saheb Kunta,0.0,0,17.370770,78.518180
107,Mallapur,0.0,0,17.350670,78.534040
156,Ramanthapur,0.0,0,17.426160,78.506860
74,Karmanghat,0.0,0,17.341389,78.465514
31,Borabanda,0.0,0,17.505990,78.304540


In [55]:
hyd_copy['cluster labels'].value_counts()

2    156
1     13
0     12
Name: cluster labels, dtype: int64

In [56]:
map_clusters= folium.Map(location=[latitude,longitude],zoom_start=11)
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(hyd_copy['Latitude'],hyd_copy['Longitude'], hyd_copy['Neighborhood'], hyd_copy['cluster labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
map_clusters.save('map_clusters.html')

# Cluster 1

In [58]:
hyd_copy.loc[hyd_copy["cluster labels"]==0]

,Neighborhood,Restaurant,cluster labels,Latitude,Longitude
134,Nawab Saheb Kunta,0.00,0,17.370770,78.518180
107,Mallapur,0.00,0,17.350670,78.534040
156,Ramanthapur,0.00,0,17.426160,78.506860
74,Karmanghat,0.00,0,17.341389,78.465514
31,Borabanda,0.00,0,17.505990,78.304540
81,Kondapur,0.25,0,17.365610,78.533050
8,Allwyn Colony,0.00,0,17.360680,78.479980
117,Mir Alam Tank,0.00,0,17.447370,78.535200
65,Jahanuma,0.00,0,17.489900,78.352200
127,Nacharam,0.00,0,17.427740,78.528920


# Cluster 2

In [59]:
hyd_copy.loc[hyd_copy["cluster labels"]==1]

,Neighborhood,Restaurant,cluster labels,Latitude,Longitude
124,Moula-Ali,0.0,1,17.400930,78.453620
130,Namalagundu,0.0,1,17.421020,78.582440
75,Karwan,0.0,1,17.354420,78.432550
51,Film Nagar,0.0,1,17.409950,78.482290
159,"Red Hills, Hyderabad",0.0,1,17.526770,78.252340
160,Rein Bazar,0.0,1,17.442320,78.496170
148,Purana pul,0.0,1,17.396420,78.488420
26,Bazarghat,0.0,1,17.402110,78.477700
9,Amberpet,0.0,1,17.503370,78.416020
6,Aghapura,0.0,1,17.387365,78.466987


# Cluster 3

In [60]:
hyd_copy.loc[hyd_copy["cluster labels"]==2]

,Neighborhood,Restaurant,cluster labels,Latitude,Longitude
110,Manovikas Nagar,0.000000,2,17.493000,78.343430
122,Moosarambagh,0.000000,2,17.465610,78.493270
123,Moti Nagar (Hyderabad),0.000000,2,17.447770,78.508730
102,Maharajgunj,0.000000,2,17.342267,78.478490
125,Musheerabad,0.000000,2,17.362015,78.470796
126,Mylargadda,0.000000,2,17.392630,78.442190
128,Nalgonda 'X' Roads,0.000000,2,17.432743,78.484542
129,Nallakunta,0.000000,2,17.355109,78.454123
101,Madinaguda,0.000000,2,17.377630,78.442370
131,"Nampally, Hyderabad",0.000000,2,17.384480,78.474420


# Observation

From the above clusters we can clearly understand that hyderabad has most numbers of Restaurants but most of Restaurants are centrally located .If we want to open a new Restaurant ,the best place is in cluster 1 or 2, as in cluster 3 we can see there are more number of Restaurants ,whereas in cluster 1 and cluster 2 very less .  so cluster 1 and 2 is the best place to open a new REstaurant.